In [2]:
import spacy
import pandas as pd
import matplotlib.pyplot as plt
from pandas import read_csv

from pathlib import Path
import random
from spacy.tokens import DocBin

## Définitions

### Entité
élement qui est spécifique et identifiable dans un texte dans le contexte NER
fait référence à des objets du monde réel tels que des personnes, des lieux, des organisations, des dates, des montants monétaires, etc. 

sont des éléments importants dans un texte, car elles portent des informations clés et contribuent à la compréhension globale du contenu.

- Personnes : "Barack Obama", "John Smith"
- Lieux : "Paris", "New York", "Mont Everest"
- Organisations : "Google", "United Nations", "Apple Inc."
- Dates : "10 août 2023", "1er janvier 2000"
- Montants monétaires : "100 dollars", "1 000 euros"
- Noms de produits : "iPhone 13", "Coca-Cola"

### NER (Named-entity recognition)

technique d'extraction d'infos qui vise à identifier et classifier les entités nommées dans un txt.

ces entités peuvent être :
- prédéfinies ou géneriques : lieux, organisations, indiations temporelles
- ou assez spécifiques : par ex dans un cv ou une recette de cuisine

l'objectif du ner c'est identifier toutes les mentions textuelles des entités nommées. On peut le faire en 2 temps : on identifie les limites du NE et ensuite son type.

On parle alors d'apprche basé sur des classificateurs. Cela signifie que l'on utilise des techniques informatiques pour enseigner à un ordinateur à identifier automatiquement les entités nommées dans le texte.

Exemple : on entraine un modele à reconnaitre differents types d'entités comme personne ou lieu. On lui montre alors de nombreux exemples de textes contenant ces entités déjà étiquetées

Un tagger/etiqueteur va etre alors un composanr qui va attribuer des labels à chaque mot dans une phrase et du coup indiquer s'il appartient ou non. Dans le cas échéant on a determine le type d'entité.


On utilise la convention "IOB", "Inside-Outside-Beginning". Elle va nous permettre d'étiqueter les mots dans un texte. Elle permet de marquer les mots qui font partie d'une entité, ceux qui commencent une nouvelle entité et ceux qui ne font pas partie d'une entité.


Exemple :  "Barack Obama a fondé la Obama Foundation en 2014 à Chicago."

"Barack" : B-PER (Beginning Person)
"Obama" : I-PER (Inside Person)
"a" : O (Outside, pas une entité)
"fondé" : O
"la" : O
"Obama" : B-ORG (Beginning Organization)
"Foundation" : I-ORG (Inside Organization)
"en" : O
"2014" : B-DATE (Beginning Date)
"à" : O
"Chicago" : B-LOC (Beginning Location)
"." : O

### Différentes méthodes/approches afin de réaliser une NER : 

- de manière classique :
 
    Comme dans la peau d'un détective qui essaie de trouver des indices dans une histoires pour identifier des personnages, des endroits et des choses spéciales

    C'est la même sauf qu'au lieu dêtre un detective le programme informatique va alors lire le texte et essayer de repérer les mots spéciaux genre lieux, marques, noms d'entreprises, personnes

    Ce programme informatique a une liste de règles préétablies, un peu comme des instructions, pour reconnaître ces noms spéciaux. Les règles préétablies sont généralement créées par des linguistes, des experts en traitement du langage naturel ou des spécialistes du domaine. Ils examinent de nombreux textes pour identifier des modèles et des caractéristiques qui peuvent indiquer la présence d'entités nommées.

    Inconvénients et Comparaison : Prenons l'exemple d'un jeu de devinettes. Imagine que tu es en train de jouer à deviner le nom d'une personne, mais les indices que tu reçois sont un peu rigides et basés sur des règles strictes. Par exemple, si quelqu'un dit "le nom que je pense commence par la lettre B et a six lettres", tu pourrais essayer des noms comme "Banane" ou "Bateau" simplement en suivant les règles, même si ce ne sont pas des noms de personnes.

    Dans le contexte de la reconnaissance d'entités nommées, la méthode classique avec des règles préétablies est un peu comme cela. Elle peut parfois donner de bons résultats si les règles correspondent bien aux modèles dans le texte, mais elle peut aussi se tromper si le texte ne suit pas exactement les règles prévues. Par exemple, si le texte utilise un surnom pour une personne comme "Bee" au lieu de "Barack", la méthode classique pourrait ne pas le reconnaître comme un nom de personne.

- grâce à ML :

    - Classification Multiclasse :
    
        Imagine qu'on a un sac rempli de différentes sortes de fruits, comme des pommes, des bananes et des oranges. Chaque fruit a ses propres caractéristiques qui le rendent unique. Maintenant, on veux apprendre à un programme à reconnaître ces fruits. Une façon de faire cela est de montrer au programme différents fruits et lui dire à chaque fois de quel fruit il s'agit => CLASSIFICATION

        La méthode de classification multiclasse traite la REN comme un jeu de devinettes. Les "devinettes" sont les différentes catégories d'entités nommées, comme les noms de personnes, les noms de lieux, etc. Le programme apprend à partir d'exemples comment identifier chaque type d'entité nommée.

        Inconvénients : Imagine qu'on essaie de comprendre une histoire complexe en lisant simplement quelques morceaux de phrases. Tu pourrais ne pas saisir toute l'histoire ou les relations entre les personnages. De même, cette méthode peut avoir du mal à saisir le sens profond du texte et à suivre comment les entités nommées sont connectées dans une séquence.

    - Modèle de Champ Aléatoire Conditionnel/Conditional Random Field (CRF) :

        Pense au CRF comme à un puzzle dans lequel tu dois placer des morceaux de texte ensemble de manière à ce qu'ils aient du sens. Le modèle CRF est comme un ensemble de règles qui t'aide à placer ces morceaux ensemble de manière logique. Cela fonctionne bien lorsque le texte suit un ordre, comme les mots dans une phrase.

        Dans la REN, chaque mot d'une phrase est comme un morceau de puzzle. Le modèle CRF aide à assembler ces morceaux en étiquetant chaque mot avec la bonne catégorie d'entité nommée. Il prend en compte les étiquettes précédentes pour s'assurer que tout s'emboîte correctement.

        Inconvénients : Imagine que tu essaies de résoudre un puzzle sans voir tous les morceaux. Cela pourrait rendre certaines parties du puzzle difficiles à résoudre. De même, le modèle CRF a du mal à prédire certaines parties du texte qui viennent plus tard dans la séquence. De plus, ce modèle peut être complexe à construire et à former, ce qui le rend moins populaire dans l'industrie.

- grâce au DL :

    Imagine que tu lis une histoire dans un livre. Quand tu lis un mot, le mot précédent et le mot suivant peuvent souvent t'aider à comprendre le sens de ce mot. Les RNN bidirectionnels fonctionnent un peu comme si tu lisais une histoire dans les deux sens à la fois : du début à la fin et de la fin au début.

    Maintenant, pense à une histoire où les personnages et les actions se développent au fil du temps. Les RNN bidirectionnels sont comme si tu lisais cette histoire tout en gardant à l'esprit les événements passés et futurs. Tu peux mieux comprendre les liens entre les personnages et les événements parce que tu as l'histoire complète dans ton esprit.

    Dans le contexte de la reconnaissance d'entités nommées (REN), chaque mot d'une phrase est comme un mot dans l'histoire. Les RNN bidirectionnels analysent le texte en prenant en compte les mots qui viennent avant et après chaque mot. Cela permet de saisir les relations complexes entre les entités nommées et les autres mots dans le texte.

    0Imagine que tu essayes de comprendre un jeu de mots croisés. Les indices dans les cases adjacentes peuvent t'aider à deviner les mots. De la même manière, les RNN bidirectionnels "devinent" quelles parties du texte sont des entités nommées en se basant sur le contexte avant et après chaque mot.


### spaCy et BERT

spaCy : spaCy est une bibliothèque de traitement du langage naturel très populaire et efficace. Elle permet de réaliser diverses tâches de traitement de texte, y compris la reconnaissance d'entités nommées (NER). spaCy offre des modèles pré-entraînés qui peuvent identifier automatiquement des entités telles que les noms de personnes, les noms de lieux, les dates, etc., dans le texte. Il permet également de créer et d'entraîner des modèles personnalisés pour des tâches spécifiques.

BERT : BERT, qui signifie "Bidirectional Encoder Representations from Transformers", est un modèle de langage pré-entraîné développé par Google. BERT est un modèle basé sur les transformers, une architecture d'apprentissage profond qui excelle dans la compréhension contextuelle du langage. BERT est conçu pour capturer les relations entre les mots dans les deux directions, ce qui lui permet de saisir des informations contextuelles riches. Il a été utilisé pour améliorer considérablement les performances dans diverses tâches de traitement de texte, y compris la REN.

CamemBERT : CamemBERT est une variante du modèle BERT spécialement adaptée à la langue française. Il a été pré-entraîné sur un grand corpus de textes en français pour mieux comprendre et représenter les spécificités de la langue française. CamemBERT est capable de saisir le sens et le contexte des mots en français, ce qui en fait un excellent choix pour les tâches de traitement de texte en français, y compris la reconnaissance d'entités nommées.

### Chunking 

Pense à une histoire que tu es en train de lire. Parfois, tu peux remarquer que des mots sont regroupés ensemble pour former des parties importantes de l'histoire. Par exemple, dans la phrase "Le chat noir dort paisiblement", tu peux regrouper les mots "le chat noir" comme une unité qui décrit quelque chose.

Dans le traitement du langage naturel (NLP), "chunking" est un processus similaire. C'est comme si nous prenions de petits morceaux d'informations, comme des mots, et les regroupions pour former des unités plus grandes et significatives. Cela nous aide à donner de la structure aux phrases en identifiant des groupes de mots qui ont une signification particulière

Chunking en NLP est un processus de regrouper de petites informations en grandes unités. L'utilisation principale du chunking est de former des groupes de "phrases nominales." Il est utilisé pour ajouter de la structure à la phrase en combinant l'étiquetage des parties du discours (POS tagging) avec des expressions régulières. Le groupe de mots résultant est appelé "chunk." Il n'y a pas de règles pré-définies pour le chunking, mais nous pouvons les créer en fonction de nos besoins. Ainsi, si nous voulons regrouper uniquement les étiquettes 'NN' (noms), nous devons utiliser un modèle.

## Code

### VISUALIZE DATA & PREPROCESS

In [5]:
data = read_csv("../datas/corpus/ner_dataset.csv", encoding = "ISO-8859-1", usecols=['Sentence #','Word','POS','Tag'])

In [6]:
sentence_number = 0 # numéros de phrase
for i, value in enumerate(data['Sentence #']): # parcourir chaque element de la colonne e 'Sentence #' 
    # i => index; value => valeur  
    if pd.notnull(value): # si la valeur actuelle est le num de la phrase / n'est pas nulle
        sentence_number += 1 # alors on += 1 sentence_number pour nvl phrase
    data.loc[i, 'Sentence #'] = sentence_number # MAJ de l'index i avec la valeur actuelle de sentence_number
data['Sentence #'].fillna(method='ffill', inplace=True)# on remplit les valeurs manquantes dans la colonne 'Sentence #' en utilisant la méthode 'ffill' (forward fill)
data['Sentence #'] = data['Sentence #'].astype(int) # MAJ en entier


C:\Users\ocean\AppData\Local\Temp\ipykernel_19324\3244459462.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['Sentence #'].fillna(method='ffill', inplace=True)# on remplit les valeurs manquantes dans la colonne 'Sentence #' en utilisant la méthode 'ffill' (forward fill)


In [7]:
data.head()

,Sentence #,Word,POS,Tag
0,1,Thousands,NNS,O
1,1,of,IN,O
2,1,demonstrators,NNS,O
3,1,have,VBP,O
4,1,marched,VBN,O


In [8]:
data['Sentence Length'] = data.groupby('Sentence #')['Word'].transform('count')
print(data['Sentence Length'])

0          24
1          24
2          24
3          24
4          24
           ..
1048570     8
1048571     8
1048572     8
1048573     8
1048574     8
Name: Sentence Length, Length: 1048575, dtype: int64


In [9]:
data.head()

,Sentence #,Word,POS,Tag,Sentence Length
0,1,Thousands,NNS,O,24
1,1,of,IN,O,24
2,1,demonstrators,NNS,O,24
3,1,have,VBP,O,24
4,1,marched,VBN,O,24


In [10]:
words = pd.DataFrame(data.groupby('Sentence #')['Word'].apply(list))
tags = data.groupby('Sentence #')['Tag'].apply(list)
words['Tag'] = tags
words['Sentence'] =  words['Word'].apply(lambda x: " ".join(map(str, x)))
words.head()

,Word,Tag,Sentence
Sentence #,,,
1,"[Thousands, of, demonstrators, have, marched, ...","[O, O, O, O, O, O, -geoB, O, O, O, O, O, B-geo...",Thousands of demonstrators have marched throug...
2,"[Families, of, soldiers, killed, in, the, conf...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",Families of soldiers killed in the conflict jo...
3,"[They, marched, from, the, Houses, of, Parliam...","[O, O, O, O, O, O, O, O, O, O, O, B-geo, I-geo...",They marched from the Houses of Parliament to ...
4,"[Police, put, the, number, of, marchers, at, 1...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","Police put the number of marchers at 10,000 wh..."
5,"[The, protest, comes, on, the, eve, of, the, a...","[O, O, O, O, O, O, O, O, O, O, O, B-geo, O, O,...",The protest comes on the eve of the annual con...


In [11]:
spacy_data = []
# pour obtenir cette forme training_data = [
#  ("Tokyo Tower is 333m tall.", [(0, 11, "BUILDING")]),
# ]
# voir https://spacy.io/usage/training#training-data

for i, row in words.iterrows():
    tokens = row["Word"]
    tags = row["Tag"]
    sentence = row["Sentence"]
    
    entities = []
    start_pos = 0  # 1er pos à 0
    
    for i in range(len(tags)):
        if tags[i] != 'O':
            end_pos = start_pos + len(str(tokens[i]))  # Calculer la der position correspond à la derniere lettre du mot à reverifier; prendre encompte les nombres en version str 
            entities.append((start_pos, end_pos, tags[i])) # stocker pour obtenir ça +/- [(0, 11, "BUILDING")]
            
            start_pos = end_pos + 1
        else : 
            start_pos += len(str(tokens[i]))
    
    data_tuple = (sentence, {"entities" : entities})
    spacy_data.append(data_tuple)

In [12]:
spacy_data[:3]

[('Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .',
  {'entities': [(42, 48, '-geoB'), (66, 70, 'B-geo'), (95, 102, 'B-gpe')]}),
 ('Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as " Bush Number One Terrorist " and " Stop the Bombings . "',
  {'entities': [(91, 95, 'B-per')]}),
 ('They marched from the Houses of Parliament to a rally in Hyde Park .',
  {'entities': [(46, 50, 'B-geo'), (51, 55, 'I-geo')]})]

### MODEL WITH SPACY

https://spacy.io/usage/training#training-data
https://towardsdatascience.com/train-ner-with-custom-training-data-using-spacy-525ce748fab7

In [13]:
model = None

In [14]:
if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

Created blank 'en' model


In [15]:
# #set up the pipeline
# if 'ner' not in nlp.pipe_names:
#     ner = nlp.add_pipe('ner')
# else:
#     ner = nlp.get_pipe('ner')

In [16]:
random.shuffle(spacy_data)
train_docs = spacy_data[:len(spacy_data) // 2]
dev_docs = spacy_data[len(spacy_data) // 2:]

In [17]:
train_docs

[("London 's benchmark Brent contract also hit a new record high of $ 61.25 .",
  {'entities': [(0, 6, 'B-geo'), (18, 23, 'B-per')]}),
 ("U.S. District Judge Federico Moreno Thursday questioned the government 's reasoning for sending back the migrants earlier this week .",
  {'entities': [(0, 4, 'B-per'),
    (5, 13, 'I-per'),
    (14, 19, 'I-per'),
    (20, 28, 'I-per'),
    (29, 35, 'I-per'),
    (36, 44, 'B-tim')]}),
 ('His first stop is China , where he is scheduled to meet with his counterpart General Cao Gangchuan and Chinese President Hu Jintao .',
  {'entities': [(14, 19, 'B-geo'),
    (63, 70, 'B-per'),
    (71, 74, 'I-per'),
    (75, 84, 'I-per'),
    (88, 95, 'B-gpe'),
    (96, 105, 'B-per'),
    (106, 108, 'I-per'),
    (109, 115, 'I-per')]}),
 ('They are believed to have been carried out by U.S. remote-controlled aircraft .',
  {'entities': [(37, 41, 'B-geo')]}),
 ("Nigeria is one of the world 's biggest oil producers , but its current refineries operate far below capacity

In [18]:
train_doc_bin  = DocBin() # conteneur pour stocker et sauvegarder efficacement les objets Doc
dev_doc_bin = DocBin()

In [19]:
from spacy.training import docs_to_json, offsets_to_biluo_tags, biluo_tags_to_spans


In [26]:
for text, annotations in train_docs:
    doc = nlp.make_doc(text)

    # for start, end, label in annotations:
    #     span = doc.char_span(start, end, label=label)
    #     print(ents)
    #     ents.append(span)
    # doc.ents = ents
    # train_doc_bin.add(doc)
    tags = offsets_to_biluo_tags(doc, annotations['entities'])
    entities = biluo_tags_to_spans(doc, tags)
    doc.ents = entities
    train_doc_bin.add(doc)
                                   
train_doc_bin.to_disk("./train.spacy")

In [27]:
for text, annotations in dev_docs:
    doc = nlp.make_doc(text)
    tags = offsets_to_biluo_tags(doc, annotations['entities'])
    entities = biluo_tags_to_spans(doc, tags)
    doc.ents = entities
    dev_doc_bin.add(doc)
                                   
dev_doc_bin.to_disk("./dev.spacy")


In [28]:
!python -m spacy init config ./config.cfg --lang en --pipeline ner --force

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [29]:
with open('config.cfg', 'r') as file:
    config_content = file.read()

print(config_content)


[paths]
train = null
dev = null
vectors = null
init_tok2vec = null

[system]
gpu_allocator = null
seed = 0

[nlp]
lang = "en"
pipeline = ["tok2vec","ner"]
batch_size = 1000
disabled = []
before_creation = null
after_creation = null
after_pipeline_creation = null
tokenizer = {"@tokenizers":"spacy.Tokenizer.v1"}

[components]

[components.ner]
factory = "ner"
incorrect_spans_key = null
moves = null
scorer = {"@scorers":"spacy.ner_scorer.v1"}
update_with_oracle_cut_size = 100

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = true
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${components.tok2vec.model.encode.width}
upstream = "*"

[components.tok2vec]
factory = "tok2vec"

[components.tok2vec.model]
@architectures = "spacy.Tok2Vec.v2"

[components.tok2vec.model.embed]
@architectures = "spacy.MultiHashEmbed.v2"
width = ${compon

In [31]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy
# lancer cette commande si pas de repertoire output 

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     50.09    0.96    0.51    7.79    0.01
  0     200         36.51   1423.26   26.70   39.88   20.06    0.27
  0     400         59.02    702.78   48.41   69.24   37.21    0.48
  0     600         90.86    732.48   51.99   56.19   48.37    0.52
  0     800        112.69    764.18   55.81   81.08   42.54    0.56
  0    1000        334.22   1011.96   64.80   76.95   55.97    0.65
  0    1200        375.49   1252.65   65.13   78.74   55.54    0.65
  0    1400        277.08   1342.70   67.93   75.86   61.50    0.68
  0    1600        241.31   139